In [1]:
cd /home/ubuntu/FedEM/

In [2]:
# Import General Libraries
import os
import argparse
import torch
import copy
import pickle
import random
import numpy as np
import pandas as pd

# Import FedEM based Libraries
from utils.utils import *
from utils.constants import *
from utils.args import *
from run_experiment import *
from models import *

# Import Transfer Attack
from transfer_attacks.Personalized_NN import *
from transfer_attacks.Params import *
from transfer_attacks.Transferer import *
from transfer_attacks.Args import *
from transfer_attacks.TA_utils import *
from transfer_attacks.Boundary_Transferer import *

In [ ]:
# Generating Empty Aggregator to be loaded 

setting = 'FedEM'

if setting == 'FedEM':
    nL = 3
else:
    nL = 1

adv_mode = True    
    
# Manually set argument parameters
args_ = Args()
args_.experiment = "cifar100"
args_.method = setting
args_.decentralized = False
args_.sampling_rate = 1.0
args_.input_dimension = None
args_.output_dimension = None
args_.n_learners= nL
args_.n_rounds = 10
args_.bz = 128
args_.local_steps = 1
args_.lr_lambda = 0
args_.lr =0.03
args_.lr_scheduler = 'multi_step'
args_.log_freq = 10
args_.device = 'cuda'
args_.optimizer = 'sgd'
args_.mu = 0
args_.communication_probability = 0.1
args_.q = 1
args_.locally_tune_clients = False
args_.seed = 1234
args_.verbose = 1
args_.save_path = 'weights/cifar/dummy/'
args_.validation = False

# Generate the dummy values here
aggregator, clients = dummy_aggregator(args_, num_user=50)

In [20]:
# Compiling Dataset from Clients
# Combine Validation Data across all clients as test
data_x = []
data_y = []

for i in range(len(clients)):
    daniloader = clients[i].test_iterator
    for (x,y,idx) in daniloader.dataset:
        data_x.append(x)
        data_y.append(y)

data_x = torch.stack(data_x)
try:
    data_y = torch.stack(data_y)        
except:
    data_y = torch.FloatTensor(data_y) 
    
dataloader = Custom_Dataloader(data_x, data_y)

In [21]:
# Import Model Weights
num_models = 50

np.set_printoptions(formatter={'float': lambda x: "{0:0.2f}".format(x)})

if setting == 'local':

    if adv_mode:
        args_.save_path ='weights/cifar100/feddef_l2/local/'
        aggregator.load_state(args_.save_path)

        model_weights = []
        weights = np.load('weights/cifar100/feddef_l2/local/train_client_weights.npy')
    else:
        args_.save_path ='weights/cifar100/first_test/local/'
        aggregator.load_state(args_.save_path)

        model_weights = []
        weights = np.load('weights/cifar100/first_test/local/train_client_weights.npy')
    
    for i in range(num_models):
        model_weights += [weights[i]]

    # Generate the weights to test on as linear combinations of the model_weights
    models_test = []

    for i in range(num_models):
        new_model = copy.deepcopy(aggregator.clients[i].learners_ensemble.learners[0].model)
        new_model.eval()
        models_test += [new_model]

elif setting == 'FedAvg':
    
    
    if adv_mode:
        args_.save_path = 'weights/cifar100/feddef_l2/fedavg/'
    else:
        args_.save_path = 'weights/cifar100/first_test/fedavg/'
    aggregator.load_state(args_.save_path)
    
    # This is where the models are stored -- one for each mixture --> learner.model for nn
    hypotheses = aggregator.global_learners_ensemble.learners

    # obtain the state dict for each of the weights 
    weights_h = []

    for h in hypotheses:
        weights_h += [h.model.state_dict()]
    
    if adv_mode:
        weights = np.load('weights/cifar100/feddef_l2/fedavg/train_client_weights.npy')
    else:
        weights = np.load('weights/cifar100/first_test/fedavg/train_client_weights.npy')
    
    # Set model weights
    model_weights = []

    for i in range(num_models):
        model_weights += [weights[i]]

    # Generate the weights to test on as linear combinations of the model_weights
    models_test = []

    for (w0) in model_weights:
        # first make the model with empty weights
        new_model = copy.deepcopy(hypotheses[0].model)
        new_model.eval()
        new_weight_dict = copy.deepcopy(weights_h[0])
        for key in weights_h[0]:
            new_weight_dict[key] = w0[0]*weights_h[0][key] 
        new_model.load_state_dict(new_weight_dict)
        models_test += [new_model]

elif setting == 'FedEM':
    
    if adv_mode:
        args_.save_path = 'weights/cifar100/feddef_l2/fedEM/'
    else:
        args_.save_path = 'weights/cifar100/first_test/fedEM/'
    aggregator.load_state(args_.save_path)
    
    # This is where the models are stored -- one for each mixture --> learner.model for nn
    hypotheses = aggregator.global_learners_ensemble.learners

    # obtain the state dict for each of the weights 
    weights_h = []

    for h in hypotheses:
        weights_h += [h.model.state_dict()]

    if adv_mode:
        weights = np.load('weights/cifar100/feddef_l2/fedEM/train_client_weights.npy')
    else:
        weights = np.load('weights/cifar100/first_test/fedEM/train_client_weights.npy')

    # Set model weights
    model_weights = []

    for i in range(num_models):
        model_weights += [weights[i]]


    # Generate the weights to test on as linear combinations of the model_weights
    models_test = []

    for (w0,w1,w2) in model_weights:
        # first make the model with empty weights
        new_model = copy.deepcopy(hypotheses[0].model)
        new_model.eval()
        new_weight_dict = copy.deepcopy(weights_h[0])
        for key in weights_h[0]:
            new_weight_dict[key] = w0*weights_h[0][key] + w1*weights_h[1][key] + w2*weights_h[2][key]
        new_model.load_state_dict(new_weight_dict)
        models_test += [new_model]

In [ ]:
model_weights2 = aggregator.clients[0].learners_ensemble.learners_weights
len(aggregator.clients[0].learners_ensemble.learners)

In [23]:
# divide models into 5 sub-groups and run 
num_groups = 1
vic_dic ={}
sub_user = 50

for i in range(num_groups):
    vic_dic[i] = range(i*sub_user, (i+1)*sub_user)

upper_logs = []

for j in range(num_groups):
    logs_adv = []

    for i in range(num_models):
        adv_dict = {}
        adv_dict['orig_acc_transfers'] = None
        adv_dict['orig_similarities'] = None
        adv_dict['adv_acc_transfers'] = None
        adv_dict['adv_similarities_target'] = None
        adv_dict['adv_similarities_untarget'] = None
        adv_dict['adv_target'] = None
        adv_dict['adv_miss'] = None
        adv_dict['metric_alignment'] = None
        adv_dict['ib_distance_legit'] = None
        adv_dict['ib_distance_adv'] = None

        logs_adv += [adv_dict]
    upper_logs += [logs_adv]

In [ ]:
rec_dict = {}

for i in range(num_groups):
    # Run Measurements for both targetted and untargeted analysis
    victim_idxs = vic_dic[i]
    custom_batch_size = 500


    for adv_idx in victim_idxs:
        print("\t Adv idx:", adv_idx)

        dataloader = load_client_data(clients = clients, c_id = adv_idx, mode = 'test') # or test/train

        batch_size = min(custom_batch_size, dataloader.y_data.shape[0])
        
        
        dataloader = load_client_data(clients = clients, c_id = adv_idx, mode = 'test') # or test/train
    
        batch_size = min(custom_batch_size, dataloader.y_data.shape[0])
#         batch_size_recs[adv_idx,:] *= batch_size

        t1 = Transferer(models_list=models_test, dataloader=dataloader)
        t1.generate_victims(victim_idxs)

        # Perform Attacks
        t1.atk_params = PGD_Params()
        t1.atk_params.set_params(batch_size=batch_size, iteration = 10,
                       target = 3, x_val_min = torch.min(data_x), x_val_max = torch.max(data_x),
                       step_size = 0.05, step_norm = "inf", eps = 4.5, eps_norm = 2)



        t1.generate_advNN(adv_idx)
        t1.generate_xadv(atk_type = "pgd")
        t1.send_to_victims(victim_idxs)

        # Log Performance
        upper_logs[i][adv_idx]['orig_acc_transfers'] = copy.deepcopy(t1.orig_acc_transfers)
        upper_logs[i][adv_idx]['orig_similarities'] = copy.deepcopy(t1.orig_similarities)
        upper_logs[i][adv_idx]['adv_acc_transfers'] = copy.deepcopy(t1.adv_acc_transfers)
        upper_logs[i][adv_idx]['adv_similarities_target'] = copy.deepcopy(t1.adv_similarities)        
        upper_logs[i][adv_idx]['adv_target'] = copy.deepcopy(t1.adv_target_hit)

        # Miss attack
        t1.atk_params.set_params(batch_size=batch_size, iteration = 20,
                       target = -1, x_val_min = torch.min(data_x), x_val_max = torch.max(data_x),
                       step_size = 0.05, step_norm = "inf", eps = 4.5, eps_norm = 2)
        t1.generate_xadv(atk_type = "pgd")
        t1.send_to_victims(victim_idxs)
        upper_logs[i][adv_idx]['adv_miss'] = copy.deepcopy(t1.adv_acc_transfers)
        upper_logs[i][adv_idx]['adv_similarities_untarget'] = copy.deepcopy(t1.adv_similarities)

In [25]:
metrics = ['orig_acc_transfers','orig_similarities','adv_acc_transfers','adv_similarities_target',
           'adv_similarities_untarget','adv_target','adv_miss']

adv_targets = []
adv_misses = []
orig_accs = []

for i in range(num_groups):
    victim_idxs = vic_dic[i]
    
    orig_acc = np.zeros([len(victim_idxs),len(victim_idxs)]) 
    orig_sim = np.zeros([len(victim_idxs),len(victim_idxs)]) 
    adv_acc = np.zeros([len(victim_idxs),len(victim_idxs)]) 
    adv_sim_target = np.zeros([len(victim_idxs),len(victim_idxs)]) 
    adv_sim_untarget = np.zeros([len(victim_idxs),len(victim_idxs)]) 
    adv_target = np.zeros([len(victim_idxs),len(victim_idxs)])
    adv_miss = np.zeros([len(victim_idxs),len(victim_idxs)]) 

    for adv_idx in range(len(victim_idxs)):
        for victim in range(len(victim_idxs)):
            orig_acc[adv_idx,victim] = upper_logs[i][victim_idxs[adv_idx]][metrics[0]][victim_idxs[victim]].data.tolist()
            orig_sim[adv_idx,victim] = upper_logs[i][victim_idxs[adv_idx]][metrics[1]][victim_idxs[victim]].data.tolist()
            adv_acc[adv_idx,victim] = upper_logs[i][victim_idxs[adv_idx]][metrics[2]][victim_idxs[victim]].data.tolist()
            adv_sim_target[adv_idx,victim] = upper_logs[i][victim_idxs[adv_idx]][metrics[3]][victim_idxs[victim]].data.tolist()
            adv_sim_untarget[adv_idx,victim] = upper_logs[i][victim_idxs[adv_idx]][metrics[4]][victim_idxs[victim]].data.tolist()
            adv_target[adv_idx,victim] = upper_logs[i][victim_idxs[adv_idx]][metrics[5]][victim_idxs[victim]].data.tolist()
            adv_miss[adv_idx,victim] = upper_logs[i][victim_idxs[adv_idx]][metrics[6]][victim_idxs[victim]].data.tolist()
            
    adv_targets += [avg_nondiag(adv_target)]
    adv_misses += [avg_nondiag(adv_miss)]
    orig_accs += [np.mean(np.diagonal(orig_acc))]

In [ ]:
print('adv_target --', 'mean:', np.mean(adv_targets), 'sd:', np.std(adv_target))
print('adv_miss --', 'mean:', np.mean(adv_misses), 'sd:', np.std(adv_miss))
print('orig_acc --', 'mean:', np.mean(orig_accs), 'sd:', np.std(orig_acc))

In [ ]:
orig_acc

In [ ]:
num_trials = 5
batch_size = 50
adv_idxs = [0,10,20,30,40]
group_list = [0,1,2,3,4]

# Create dataloader from validation dataset that allows for diverse batch size
dataloader = Custom_Dataloader(data_x, data_y)
num_model2 = 3

dmls = []
dma = []
dmae = []

for i in group_list:
    victim_idxs = vic_dic[i]
    adv_idx = adv_idxs[i]

    dists_measure_legit = np.zeros([num_trials, num_model2-1])
    dists_measure_adv = np.zeros([num_trials, num_model2-1])
    dists_measure_adv_ensemble = np.zeros([num_trials, num_model2-1])


#     for j in range(num_model2):
    for i in range(num_trials):
        print("num_trial:", i)
        t1 = Boundary_Transferer(models_list=models_test[victim_idxs[0]:(victim_idxs[-1]+1)], dataloader=dataloader)
        t1.base_nn_idx = 0
        t1.victim_idx = list(range(1, num_model2))

        t1.atk_params = PGD_Params()
        t1.atk_params.set_params(batch_size=500, iteration = 30,
                       target = -1, x_val_min = torch.min(data_x), x_val_max = torch.max(data_x),
                       step_size = 0.05, step_norm = "inf", eps = 3, eps_norm = 2)
        t1.set_adv_NN(t1.base_nn_idx)

        base_ep_legit, victim_eps_legit = t1.legitimate_direction(batch_size=batch_size, ep_granularity = 0.5, 
                                                                  rep_padding = 200, new_point = True,print_res = False)

        base_ep_adv, victim_eps_adv = t1.adversarial_direction(ep_granularity = 0.5, 
                                                                  rep_padding = 200, new_point = False,print_res = False)

        idx = 0
        for key, value in victim_eps_legit.items():
            dists_measure_legit[i,idx] = np.abs(base_ep_legit-value)
            idx+=1

        idx = 0
        for key, value in victim_eps_adv.items():
            dists_measure_adv[i,idx] = np.abs(base_ep_adv - value)
            idx+=1
                
    dmls += [dists_measure_legit]
    dma += [dists_measure_adv]
    

In [ ]:
print('legit mean:', np.mean(np.average(dmls,axis=1)))
print('legit std:', np.std(np.average(dmls,axis=1))
print('adv mean:',np.mean(np.average(dma,axis=1)))
print('adv std:', np.std(np.average(dma,axis=1)))